# Predictive Maintenance 

This notebook addresses system failure detection in a highly imbalanced dataset using class-weighted models and threshold tuning.

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, f1_score, recall_score, accuracy_score
import joblib

# Set random seed
np.random.seed(42)

In [23]:
# Load data
df = pd.read_csv('system_logs_ready_min.csv')
print(f"Dataset Shape: {df.shape}")
print("\nTarget Distribution (Original):")
print(df['failure'].value_counts())

Dataset Shape: (876100, 7)

Target Distribution (Original):
failure
0    875381
1       719
Name: count, dtype: int64


In [24]:
# Drop timestamp
if 'timestamp' in df.columns:
    df_clean = df.drop(columns=['timestamp'])
else:
    df_clean = df.copy()
df_clean = df_clean.dropna()

# BALANCING STRATEGY: 
# Total failures are ~719. We will sample 5,000 normal cases to maintain a strong failure signal.
df_failures = df_clean[df_clean['failure'] == 1]
df_normal = df_clean[df_clean['failure'] == 0]

sample_normal = df_normal.sample(min(5000, len(df_normal)), random_state=42)
df_train_pool = pd.concat([df_failures, sample_normal]).sample(frac=1, random_state=42)

X = df_train_pool.drop(columns=['failure'])
y = df_train_pool['failure']
print(f"Balanced Pool Shape: {len(df_train_pool)} (Failures: {len(df_failures)})")

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Stratified Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
print(f"Training set: {len(y_train)} rows | Testing set: {len(y_test)} rows")

Balanced Pool Shape: 5719 (Failures: 719)
Training set: 4575 rows | Testing set: 1144 rows


In [25]:
# Training models with Balanced Class Weights
print("Training models...")

# Logistic Regression
log_reg = LogisticRegression(class_weight='balanced', random_state=42)
log_reg.fit(X_train, y_train)

# Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf_clf.fit(X_train, y_train)

# Linear SVM
svm_clf = LinearSVC(dual=False, class_weight='balanced', random_state=42)
svm_clf.fit(X_train, y_train)

print("Done.")

Training models...
Done.


In [26]:
models = {'Logistic Regression': log_reg, 'Random Forest': rf_clf, 'SVM': svm_clf}
results = {}

print("Model Evaluation (Default Threshold 0.5):\n")
for name, model in models.items():
    y_pred = model.predict(X_test)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)
    
    results[name] = {'recall': recall, 'f1': f1, 'accuracy': acc}
    
    print(f"=== {name} ===")
    print(f"Recall: {recall:.4f} | F1: {f1:.4f} | Accuracy: {acc:.4f}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("-" * 30)

Model Evaluation (Default Threshold 0.5):

=== Logistic Regression ===
Recall: 0.5972 | F1: 0.2960 | Accuracy: 0.6425
Confusion Matrix:
 [[649 351]
 [ 58  86]]
------------------------------
=== Random Forest ===
Recall: 0.0556 | F1: 0.0964 | Accuracy: 0.8689
Confusion Matrix:
 [[986  14]
 [136   8]]
------------------------------
=== SVM ===
Recall: 0.5972 | F1: 0.2966 | Accuracy: 0.6434
Confusion Matrix:
 [[650 350]
 [ 58  86]]
------------------------------


In [27]:
# THRESHOLD TUNING (Critical for catching failures early)
y_scores = log_reg.decision_function(X_test)
y_probs = 1 / (1 + np.exp(-y_scores))

print("Effect of lowering Decision Threshold (Logistic Regression):")
for t in [0.5, 0.4, 0.3, 0.2, 0.1]:
    y_pred_t = (y_probs >= t).astype(int)
    r = recall_score(y_test, y_pred_t)
    f = f1_score(y_test, y_pred_t)
    print(f"Threshold: {t:.1f} -> Recall: {r:.4f} | F1-Score: {f:.4f}")

Effect of lowering Decision Threshold (Logistic Regression):
Threshold: 0.5 -> Recall: 0.5972 | F1-Score: 0.2960
Threshold: 0.4 -> Recall: 0.8056 | F1-Score: 0.2886
Threshold: 0.3 -> Recall: 0.9167 | F1-Score: 0.2573
Threshold: 0.2 -> Recall: 0.9792 | F1-Score: 0.2384
Threshold: 0.1 -> Recall: 1.0000 | F1-Score: 0.2255


In [28]:
# Select Best Model based on Recall
best_model_name = max(results, key=lambda x: results[x]['recall'])
print(f"Selected Model for Predictive Maintenance: {best_model_name}")

Selected Model for Predictive Maintenance: Logistic Regression


In [29]:
# Save artifacts
best_model = models[best_model_name]
joblib.dump(best_model, 'best_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
print(f"Artifacts saved. Best Model: {best_model_name}")

Artifacts saved. Best Model: Logistic Regression
